# Emojify! 



In [1]:
import numpy as np
from emo_utils import *
import emoji
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import numpy as np
np.random.seed(0)
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
np.random.seed(1)

Using TensorFlow backend.


In [4]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)]
    return Y

In [5]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r',encoding='UTF-8') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word =''.join(line[0])
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=float)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

def read_csv(filename = 'data/emojify_data.csv'):
    phrase = []
    emoji = []

    with open (filename) as csvDataFile:
        csvReader = csv.reader(csvDataFile)

        for row in csvReader:
            phrase.append(row[0])
            emoji.append(row[1])

    X = np.asarray(phrase)
    Y = np.asarray(emoji, dtype=int)

    return X, Y

In [6]:
X_train, Y_train = read_csv('data/train_emoji.csv')
X_test, Y_test = read_csv('data/test.csv')
maxLen = len(max(X_train, key=len).split())
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('data/glove.6B.200d.txt')

In [7]:
def sentences_to_indices(X, word_to_index, max_len):
    
    
    m = X.shape[0]                                   # number of training examples
    
    X_indices = np.zeros((m,max_len))
    
    for i in range(m):                               # loop over training examples
        
        sentence_words =(X[i].lower()).split()
        
        j = 0
        for w in sentence_words:
            X_indices[i, j] = word_to_index[w]
            j = j + 1
            
    
    return X_indices

In [8]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):

    
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["cucumber"].shape[0]      # define dimensionality of your GloVe word vectors (= 50)
    
    emb_matrix = np.zeros((vocab_len, emb_dim))
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]
    embedding_layer = Embedding(vocab_len, emb_dim, trainable=False)

    embedding_layer.build((None,))
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [9]:
def Emojify(input_shape, word_to_vec_map, word_to_index):
  
    
    sentence_indices = Input(input_shape, dtype='int32')
    
    embedding_layer =  pretrained_embedding_layer(word_to_vec_map, word_to_index)
    embeddings = embedding_layer(sentence_indices)   
    X = LSTM(128, return_sequences=True)(embeddings)
    X = Dropout(0.5)(X)
    X = LSTM(128, return_sequences=False)(X)
    X = Dropout(0.5)(X)
    X = Dense(5)(X)
    X = Activation('softmax')(X)
    
    model = Model(sentence_indices,X)
    

    return model

In [10]:
model = Emojify((maxLen,), word_to_vec_map, word_to_index)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 10, 200)           80000200  
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 128)           168448    
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 645       
__________

In [11]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [12]:
X_train_indices = sentences_to_indices(X_train, word_to_index, maxLen)
Y_train_oh = convert_to_one_hot(Y_train, C = 5)

In [13]:
model.fit(X_train_indices, Y_train_oh, epochs = 50, batch_size = 32, shuffle=True)

Epoch 1/50
132/132 [==============================] - 10s 72ms/step - loss: 1.5799 - acc: 0.3333
Epoch 2/50
132/132 [==============================] - 0s 1ms/step - loss: 1.5153 - acc: 0.2955
Epoch 3/50
132/132 [==============================] - 0s 1ms/step - loss: 1.4129 - acc: 0.4318
Epoch 4/50
132/132 [==============================] - 0s 1ms/step - loss: 1.3115 - acc: 0.5455
Epoch 5/50
132/132 [==============================] - 0s 1ms/step - loss: 1.1409 - acc: 0.6439
Epoch 6/50
132/132 [==============================] - 0s 2ms/step - loss: 0.9645 - acc: 0.6515
Epoch 7/50
132/132 [==============================] - 0s 1ms/step - loss: 0.7477 - acc: 0.7955
Epoch 8/50
132/132 [==============================] - 0s 2ms/step - loss: 0.6573 - acc: 0.7652
Epoch 9/50
132/132 [==============================] - 0s 1ms/step - loss: 0.5826 - acc: 0.7879
Epoch 10/50
132/132 [==============================] - 0s 2ms/step - loss: 0.4713 - acc: 0.8485
Epoch 11/50
132/132 [==========================

In [14]:
X_test_indices = sentences_to_indices(X_test, word_to_index, max_len = maxLen)
Y_test_oh = convert_to_one_hot(Y_test, C = 5)
loss, acc = model.evaluate(X_test_indices, Y_test_oh)
print()
print("Test accuracy = ", acc)

56/56 [==============================] - 0s 2ms/step

Test accuracy =  0.821428562914


In [15]:
# This code allows you to see the mislabelled examples
C = 5
y_test_oh = np.eye(C)[Y_test.reshape(-1)]
X_test_indices = sentences_to_indices(X_test, word_to_index, maxLen)
pred = model.predict(X_test_indices)
for i in range(len(X_test)):
    x = X_test_indices
    num = np.argmax(pred[i])
    if(num != Y_test[i]):
        print('Expected emoji:'+ label_to_emoji(Y_test[i]) + ' prediction: '+ X_test[i] + label_to_emoji(num).strip())

Expected emoji:😄 prediction: he is a good friend	❤️
Expected emoji:😄 prediction: We had such a lovely dinner tonight	❤️
Expected emoji:😞 prediction: This girl is messing with me	❤️
Expected emoji:😄 prediction: Congratulation for having a baby	❤️
Expected emoji:😄 prediction: you brighten my day	😞
Expected emoji:😞 prediction: she is a bully	❤️
Expected emoji:😞 prediction: My life is so boring	😄
Expected emoji:😄 prediction: will you be my valentine	😞
Expected emoji:😞 prediction: go away	⚾
Expected emoji:❤️ prediction: family is all I have	😞


In [21]:
# Change the sentence below to see your prediction. Make sure all the words are in the Glove embeddings.  
x_test = np.array(['not feeling happy'])
X_test_indices = sentences_to_indices(x_test, word_to_index, maxLen)
print(x_test[0] +' '+  label_to_emoji(np.argmax(model.predict(X_test_indices))))

not feeling happy 😞
